## Scrapping urbania

The goal of this notebook is to scrape data from https://urbania.pe/ and store it on a mongo databse

### Libraries import

In [1]:
import math
import numpy as np
import pymongo
import re 
import urllib.request as urllib2
import datetime

from time import sleep
from bs4 import BeautifulSoup

from constants import *

In [2]:
# bsae urls to use
base_url = 'https://urbania.pe{}'
apartments_url = 'https://urbania.pe/buscar/venta-de-departamentos'


### Define database connections

In [5]:
uri = "mongodb://{}:{}@{}/{}".format(username, password, hostlist, database)
client = pymongo.MongoClient(uri)
db = client['urbania']

In [6]:
list(db['admin'].find())

[{'_id': ObjectId('5deb3aba3dadd940d2148a36'), 'name': 'gonzalo'},
 {'_id': ObjectId('5dec891299df02824e747081'), 'name': 'jay'}]

In [7]:
apartmentsdb = db['apartments']

### Here we put the logic to get the data

In [13]:
def getSoupObj(url):
    sleep(1.5)
    try:
        r = urllib2.urlopen(url)
    except UnicodeEncodeError as e:
        return None
    html = r.read()
    htmltext = html.decode('ISO-8859-2')
    return BeautifulSoup(htmltext, 'lxml')

In [14]:
tihs = 10000
count = 1
while count < 5:
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = urllib2.Request(apartments_url, headers=hdr)
    apartmentsBsObj = getSoupObj(req)
    
    objs = apartmentsBsObj.findAll('article', {'class': 'b-card-item js-card-item b-card-item--proyecto-destacado'})
#     objsSeen = apartmentsBsObj
    print('In page {} we found {} apartments'.format(count, len(objs)))
    for obj in objs:
        
#         the objA is the object with <a> label in obj
        objA = obj.find('a')
        if objA is None: continue
            
#         objA = objA.text
        apartment = {}
        
        apartment['_id'] = obj['data-identifier']
        apartment['url'] = objA['href']
#         print(type(obj), type(objA))
#         print(obj['data-identifier'])
        get_apartment(apartment)
#         print(apartment)
        break
    count +=1
    break

In page 1 we found 14 apartments


EN CONSTRUCCIĂN
ENTREGA DICIEMBRE 2020


Tipos Dptos.
Residencial


Dpto.
Lima


Provincia
Lima


Distrito
San Borja


Urb.
Corpac


Referencia
A POCOS METROS DE AV. GUARDIA CIVIL Y AV.  SAN BORJA NORTE, RODEADO DE ĂREAS VERDES




In [10]:
def get_apartment(apartment):
    
    apartment_url = base_url.format(apartment['url'])
#     print(apartment_url)
    req = urllib2.Request(apartment_url)
    apartmentObj = getSoupObj(req)
#     print(apartmentObj)
    
    if apartmentObj == -1:
        return
    
    get_text(apartment, apartmentObj, 'title', 'e-title e-subtitle--with-logo', 'h1')
    get_text(apartment, apartmentObj, 'address', 'e-subtitle e-subtitle--with-logo', 'p')
    get_text(apartment, apartmentObj, 'features', 'e-features', 'p')
    get_text(apartment, apartmentObj, 'intial_price', 'e-totalPrice', 'p')
    print(get_text(apartment, apartmentObj, 'intial_price', 'b-wrap-interbank u-flex-wrap', 'div', False))
    
    
#     print(apartment)
#     print(address.text)


In [11]:
def get_text(obj, objBs, feature, tag, marker, inplace=True):
    new_feature = objBs.find(marker, {'class': tag})
    obj[feature] = new_feature.text
    if inplace ==False:
        return new_feature.text

In [ ]:
def get_real_estate(real_estate):
    